<a href="https://colab.research.google.com/github/IrfanKhalid/ForexExchangePrediction/blob/master/ForexExchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn as sk

In [0]:
from google.colab import drive
drive.mount('content/drive')

In [0]:
Forex=pd.read_csv(data)